In [1]:
import livef1

In [2]:
season = livef1.get_season(2025)

In [3]:
schedule = season.get_schedule()

schedule

,Meeting Shortname,Meeting Offname,Session Name,Start Date,End Date
0,2025 F1 Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Day 1,2025-02-26 07:00:00,2025-02-26 16:00:00
1,2025 F1 Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Day 2,2025-02-27 07:00:00,2025-02-27 16:00:00
2,2025 F1 Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Day 3,2025-02-28 07:00:00,2025-02-28 16:00:00
3,Australian,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Practice 1,2025-03-14 01:30:00,2025-03-14 02:30:00
4,Australian,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Practice 2,2025-03-14 05:00:00,2025-03-14 06:00:00
...,...,...,...,...,...
118,Abu Dhabi,FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX ...,Practice 1,2025-12-05 09:30:00,2025-12-05 10:30:00
119,Abu Dhabi,FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX ...,Practice 2,2025-12-05 13:00:00,2025-12-05 14:00:00
120,Abu Dhabi,FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX ...,Practice 3,2025-12-06 10:30:00,2025-12-06 11:30:00
121,Abu Dhabi,FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX ...,Qualifying,2025-12-06 14:00:00,2025-12-06 15:00:00


In [4]:
schedule.to_csv("2025_schedule.csv", index=False)

In [ ]:
livef1.get_meeting(2025, circuit)

In [24]:
for _, row in schedule.groupby("Meeting Offname").first().iterrows():
    print(row)
    circuit = row["Meeting Circuit Shortname"]

    for year in [2025,2024,2023,2022,2021,2020,2019,2018]:
        try:
            m = livef1.get_meeting(year, circuit)
            break
        except:
            pass

Meeting Circuit Shortname                  Spain
Session Name                          Practice 1
Start Date                   2025-05-30 11:30:00
End Date                     2025-05-30 12:30:00
Name: FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025, dtype: object


16:07:58 - The identifier couldn't be found.
16:07:58 - LiveF1Error - 
The searched query 'Spain' not found in the table. Did you mean one of these :

	Meeting Offname : FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025
	Meeting Name : Spanish Grand Prix
	Meeting Circuit Shortname : Catalunya
	> Suggested search queries : [['aramco', 'gran', 'premio', 'de', 'españa', '2025'], ['spanish'], ['catalunya']]

	Meeting Offname : FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025
	Meeting Name : Bahrain Grand Prix
	Meeting Circuit Shortname : Sakhir
	> Suggested search queries : [['gulf', 'air', 'bahrain', '2025'], ['bahrain'], ['sakhir']]

	Meeting Offname : FORMULA 1 ARAMCO PRE-SEASON TESTING 2025
	Meeting Name : Pre-Season Testing
	Meeting Circuit Shortname : Sakhir
	> Suggested search queries : [['aramco', 'pre', 'season', 'testing', '2025'], ['pre', 'season', 'testing'], ['sakhir']]


16:08:00 - The identifier couldn't be found.
16:08:00 - LiveF1Error - 
The searched query 'Spain' not found in the tab

Meeting Circuit Shortname                 Sakhir
Session Name                               Day 1
Start Date                   2025-02-26 07:00:00
End Date                     2025-02-26 16:00:00
Name: FORMULA 1 ARAMCO PRE-SEASON TESTING 2025, dtype: object


16:08:13 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025
	Meeting Name : Bahrain Grand Prix
	Meeting Circuit Shortname : Sakhir
16:08:13 - Got the meeting.


Meeting Circuit Shortname         Emilia-Romagna
Session Name                          Practice 1
Start Date                   2025-05-16 11:30:00
End Date                     2025-05-16 12:30:00
Name: FORMULA 1 AWS GRAN PREMIO DEL MADE IN ITALY E DELL'EMILIA-ROMAGNA 2025, dtype: object


...

16:10:05 - The identifier couldn't be found.
16:10:05 - LiveF1Error - 
The searched query 'Singapore' not found in the table. Did you mean one of these :

	Meeting Offname : FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025
	Meeting Name : Bahrain Grand Prix
	Meeting Circuit Shortname : Sakhir
	> Suggested search queries : [['gulf', 'air', 'bahrain', '2025'], ['bahrain'], ['sakhir']]

	Meeting Offname : FORMULA 1 ARAMCO PRE-SEASON TESTING 2025
	Meeting Name : Pre-Season Testing
	Meeting Circuit Shortname : Sakhir
	> Suggested search queries : [['aramco', 'pre', 'season', 'testing', '2025'], ['pre', 'season', 'testing'], ['sakhir']]

	Meeting Offname : FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025
	Meeting Name : Monaco Grand Prix
	Meeting Circuit Shortname : Monte Carlo
	> Suggested search queries : [['tag', 'heuer', 'de', 'monaco', '2025'], ['monaco'], ['monte', 'carlo']]


16:10:06 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 SINGAPORE AIRLINES SINGAPORE GRAND PRIX 2024
	Meet

Meeting Circuit Shortname           Saudi Arabia
Session Name                          Practice 1
Start Date                   2025-04-18 13:30:00
End Date                     2025-04-18 14:30:00
Name: FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025, dtype: object


16:10:08 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025
	Meeting Name : Saudi Arabian Grand Prix
	Meeting Circuit Shortname : Jeddah
16:10:08 - Got the meeting.


Meeting Circuit Shortname                 Monaco
Session Name                          Practice 1
Start Date                   2025-05-23 11:30:00
End Date                     2025-05-23 12:30:00
Name: FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025, dtype: object


16:10:09 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025
	Meeting Name : Monaco Grand Prix
	Meeting Circuit Shortname : Monte Carlo
16:10:09 - Got the meeting.


In [17]:
for c in schedule["Meeting Circuit Shortname"].unique():
    print(c)
    for year in [2024,2023,2022,2021,2020,2019,2018]:
        try:
            m = livef1.get_meeting(year, c)
            break
        except: 
            print(f"Not found in {year}")

Sakhir


14:41:35 - The identifier couldn't be found.
14:41:35 - LiveF1Error - 
The searched query 'Sakhir' not found in the table. Did you mean one of these :

	Meeting Offname : FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2024
	Meeting Name : Qatar Grand Prix
	Meeting Circuit Shortname : Lusail
	> Suggested search queries : [['qatar', 'airways', 'qatar', '2024'], ['qatar'], ['lusail']]

	Meeting Offname : FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2024
	Meeting Name : Spanish Grand Prix
	Meeting Circuit Shortname : Catalunya
	> Suggested search queries : [['aramco', 'gran', 'premio', 'de', 'españa', '2024'], ['spanish'], ['catalunya']]

	Meeting Offname : FORMULA 1 SINGAPORE AIRLINES SINGAPORE GRAND PRIX 2024
	Meeting Name : Singapore Grand Prix
	Meeting Circuit Shortname : Singapore
	> Suggested search queries : [['singapore', 'airlines', 'singapore', '2024'], ['singapore'], ['singapore']]




Not found in 2024


14:41:38 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023
	Meeting Name : Bahrain Grand Prix
	Meeting Circuit Shortname : Sakhir
14:41:38 - Got the meeting.


Australia


...

14:43:44 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 HEINEKEN SILVER LAS VEGAS GRAND PRIX 2024
	Meeting Name : Las Vegas Grand Prix
	Meeting Circuit Shortname : Las Vegas
14:43:44 - Got the meeting.


Qatar


14:43:46 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2024
	Meeting Name : Qatar Grand Prix
	Meeting Circuit Shortname : Lusail
14:43:46 - Got the meeting.


Abu Dhabi


14:43:47 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX 2024
	Meeting Name : Abu Dhabi Grand Prix
	Meeting Circuit Shortname : Yas Marina Circuit
14:43:47 - Got the meeting.
